In [ ]:
import os
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt
import imutils
from imutils.video import VideoStream
from imutils import paths
from tensorflow.keras.models import Model
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## Creation of the predictive model

In [ ]:
init_learn_rate = 1e-4
epochs = 20
batch = 32
directory = "/content/dataset"
categories = ["with_mask", "without_mask"]

In [ ]:
print("Loading dataset images")
data = []
labels = []

In [ ]:
for category in categories:
    path = os.path.join(directory, category)
    
    for image in os.listdir(path):
        img_path = os.path.join(path, image)
        nthimg = load_img(img_path, target_size=(224, 224))
        nthimg = img_to_array(image)
        nthimg = preprocess_input(image)
        
        data.append(nthimg)
        labels.append(category) 

In [ ]:
# encoding labels
labels = LabelBinarizer().fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)

In [ ]:
(train_x, test_x, train_y, test_y) = train_test_split(
data, labels, test_size=0.20, stratify=labels, random_state=42
)

In [ ]:
# training image generator for data augmentation
d_aug = ImageDataGenerator(
    rotation_range=20, zoom_range=0.15, width_shift_range=0.2,
    height_shift_range=0.2, shear_range=0.15, 
    horizontal_flip=True, fill_mode="nearest"
)

In [ ]:
# Loading MobileNetV2
model_base = MobileNetV2(weights="imagenet", include_top=False,
                        input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
# model head
model_head = AveragePooling2D(pool_size=(7, 7))(model_base.output)
model_head = Flatten(name="flatten")(model_head)
model_head = Dense(128, activation="relu")(model_head)
model_head = Dropout(0.5)(model_head)
model_head = Dense(2, activation="softmax")(model_head)

In [ ]:
# model building
model = Model(inputs=model_base.input, outputs=model_head)

In [ ]:
for layer in model_base.layers:
    layer.trainable = False

In [ ]:
# Compiling model
opt = Adam(lr=init_learn_rate, decay=init_learn_rate / epochs)
model.compile(
    loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"]
)

In [ ]:
batch_s = 32
head_train = model.fit(
    d_aug.flow(train_x, train_y, batch_size=batch_s),
    steps_per_epoch=len(train_x) // batch_s,
    validation_data=(test_x, test_y),
    validation_steps=len(test_x) // batch_s,
    epochs = epochs
)

In [ ]:
# predictions on testing set
pred_inds = model.predict(test_x, batch_size=batch_s)

# label index with largest predicted probability
pred_inds = np.argmax(pred_inds, axis=1)

In [ ]:
# classification report
classification_report(test_y.argmax(axis=1), pred_inds,
                     target_names=lb.classes_)

In [ ]:
# serialize the model to disk
model.save("mask_detector.model", save_format="h5")

In [ ]:
# training loss and accuracy plot
num = epochs
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, num), head_train.history["loss"], label="train_loss")
plt.plot(np.arange(0, num), head_train.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, num), head_train.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, num), head_train.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch number")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")